<a href="https://colab.research.google.com/github/narsym/Movie-Q-A-chatbot/blob/main/Q_and_A_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=b72f9adcdbbf6f1372ca914554ba290b79e9f0c61ab33725f58163d6158adc38
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [3]:
pip install gensim

In [4]:
!pip install torch
!pip install transformers

     |████████████████████████████████| 1.8MB 9.3MB/s 
     |████████████████████████████████| 3.2MB 43.1MB/s 
     |████████████████████████████████| 890kB 41.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e04f0effac52d3f4e027f278e02e09b40c382bb11281e35d4c1d6c782da45cf7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:

from gensim.summarization.bm25 import BM25
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

class QuestionProcessor:


    def __init__(self, nlp):
        self.pos = ["NOUN", "PROPN", "ADJ"]
        self.nlp = nlp


    def process(self, text):
        tokens = self.nlp(text)
        return ' '.join(token.text for token in tokens if token.pos_ in self.pos)

class ContextRetriever:

    def __init__(self, nlp, numberOfResults):
        self.nlp = nlp
        self.numberOfResults = numberOfResults

    def tokenize(self, sentence):
        return [token.lemma_ for token in self.nlp(sentence)]


    def getContext(self, sentences, question):
        documents = []
        for sent in sentences:
            documents.append(self.tokenize(sent))

        bm25 = BM25(documents)

        scores = bm25.get_scores(self.tokenize(question), 1)
        results = {}
        for index, score in enumerate(scores):
            results[index] = score

        sorted_results = {k: v for k, v in sorted(results.items(), key=lambda item: item[1], reverse=True)}
        results_list = list(sorted_results.keys())
        final_results = results_list if len(results_list) < self.numberOfResults else results_list[:self.numberOfResults]
        questionContext = ""
        for final_result in final_results:
            questionContext = questionContext + " ".join(documents[final_result])
        return questionContext



In [19]:
import torch
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering


class AnswerRetriever:

    def getAnswer(self, question, questionContext):
        distilBertTokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', return_token_type_ids=True)
        distilBertForQuestionAnswering = DistilBertForQuestionAnswering.from_pretrained(
            'distilbert-base-uncased-distilled-squad')

        encodings = distilBertTokenizer.encode_plus(question, questionContext)

        inputIds, attentionMask = encodings["input_ids"], encodings["attention_mask"]

        scoresStart, scoresEnd = distilBertForQuestionAnswering(torch.tensor([inputIds]),
                                                                attention_mask=torch.tensor([attentionMask]), return_dict = False)

        tokens = inputIds[torch.argmax(scoresStart): torch.argmax(scoresEnd) + 1]
        answerTokens = distilBertTokenizer.convert_ids_to_tokens(tokens, skip_special_tokens=True)
        return distilBertTokenizer.convert_tokens_to_string(answerTokens)

In [5]:
import time

In [6]:
import wikipedia
import spacy
import torch
from gensim.summarization.bm25 import BM25

In [7]:
def search_article(movie_name):
  results = wikipedia.search(movie_name)
  return results
search_article('interstellar')

['Interstellar',
 'Interstellar (film)',
 'Interstellar travel',
 'Interstellar medium',
 'Cosmic dust',
 'Interstellar object',
 'Interstellar (soundtrack)',
 'The Science of Interstellar',
 'Solar System',
 'Interstellar probe']

In [8]:
content = ''
def download_article(movie_name):
  global content
  page = wikipedia.page(movie_name)
  content = page.content

download_article('Interstellar (film)')
content

'Interstellar is a 2014 epic science fiction film directed and produced by Christopher Nolan. It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, John Lithgow, Michael Caine, and Matt Damon. Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.\nBrothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007. Christopher produced Interstellar with his wife, Emma Thomas, through their production company Syncopy, and with Lynda Obst through Lynda Obst Productions. Caltech theoretical physicist and 2017 Nobel laureate in Physics Kip Thorne was an executive producer, acted as scientific consultant, and wrote a tie-in book, The Science of Interstellar. Paramount Pictures, Warner Bros., and Legendary Pictures co-financed the film. Cinematographer Hoyte van Hoytema sho

In [38]:
def ask_question(question, content):
  nlp = spacy.load('en_core_web_sm')
  a = time.perf_counter()
  nlp.add_pipe(nlp.create_pipe('sentencizer'))
  doc = nlp(content)
  sentences = [sent.string.strip() for sent in doc.sents]
  questionProcessor = QuestionProcessor(nlp)
  contextRetriever = ContextRetriever(nlp, 10)
  answerRetriever = AnswerRetriever()

  originalQuestion = question
  questionContext = contextRetriever.getContext(sentences, questionProcessor.process(originalQuestion))
  answer = answerRetriever.getAnswer(originalQuestion, questionContext)
  print('time taken to answer:', time.perf_counter() - a)
  return answer

print('Answer is:', ask_question('Which stars are there?', content))

time taken to answer: 9.942099168999903
Answer is: matthew mcconaughey , anne hathaway , jessica chastain , bill irwin , ellen burstyn , john lithgow , michael caine , and matt damon


taking approx 10 sec to answer